# Libraries

In [1]:
import numpy as np
import os, cv2
from matplotlib import pyplot as plt

from pcdet.utils import common_utils
from pcdet.datasets import build_dataloader
from pcdet.config import cfg, log_config_to_file, cfg_from_list, cfg_from_yaml_file

# Move to the root directory
if not 'workdir' in globals():
    workdir = os.getcwd()
    workdir = "/".join(workdir.split('/')[:-1])
%cd "$workdir"

# Create cache folder
os.makedirs("cache", exist_ok=True)

/home/thuync/Workspace/OpenLidarPerceptron


# Build dataset

In [4]:
batch_size = 1
workers = 4
dist_train = False
cfg_file = "tools/cfgs/kitti_models/pointpillar_gn.yaml"
log_file = "cache/dataset_debug.txt"

cfg_from_yaml_file(cfg_file, cfg)
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

In [5]:
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=batch_size,
    dist=dist_train, workers=workers,
    logger=logger,
    training=True,
)

print("Dataset size:", len(train_set))
print("Classnames:", train_set.class_names)

2020-07-02 09:12:46,729   INFO  Database filter by min points Car: 14357 => 13532
2020-07-02 09:12:46,729   INFO  Database filter by min points Car: 14357 => 13532
2020-07-02 09:12:46,731   INFO  Database filter by min points Pedestrian: 2207 => 2168
2020-07-02 09:12:46,731   INFO  Database filter by min points Pedestrian: 2207 => 2168
2020-07-02 09:12:46,732   INFO  Database filter by min points Cyclist: 734 => 705
2020-07-02 09:12:46,732   INFO  Database filter by min points Cyclist: 734 => 705
2020-07-02 09:12:46,760   INFO  Database filter by difficulty Car: 13532 => 10759
2020-07-02 09:12:46,760   INFO  Database filter by difficulty Car: 13532 => 10759
2020-07-02 09:12:46,766   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2020-07-02 09:12:46,766   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2020-07-02 09:12:46,768   INFO  Database filter by difficulty Cyclist: 705 => 581
2020-07-02 09:12:46,768   INFO  Database filter by difficulty Cyclist: 705 => 

Dataset size: 3712
Classnames: ['Car', 'Pedestrian', 'Cyclist']


# Visualize

In [13]:
idx = 0
data_batch = train_set[idx]
print("Batch components:", data_batch.keys())

points = data_batch['points']
frame_id = data_batch['frame_id']
gt_boxes = data_batch['gt_boxes']
use_lead_xyz = data_batch['use_lead_xyz']
voxels = data_batch['voxels']
voxel_coords = data_batch['voxel_coords']
voxel_num_points = data_batch['voxel_num_points']
image_shape = data_batch['image_shape']

for key, val in data_batch.items():
    if isinstance(val, (str, bool)):
        print("{}: {}".format(key, val))
    else:
        print("{}: {}".format(key, val.shape))

Batch components: dict_keys(['points', 'frame_id', 'gt_boxes', 'use_lead_xyz', 'voxels', 'voxel_coords', 'voxel_num_points', 'image_shape'])
points: (24687, 4)
frame_id: 000000
gt_boxes: (38, 8)
use_lead_xyz: True
voxels: (4228, 32, 4)
voxel_coords: (4228, 3)
voxel_num_points: (4228,)
image_shape: (2,)


In [30]:
point_idx = 100
print(points[point_idx])
print(voxel_coords[point_idx])
print(voxels[point_idx])

[16.957018   3.4763894 -1.0700533  0.12     ]
[  0 301  54]
[[ 8.73306    8.504156  -1.5780946  0.31     ]
 [ 8.654741   8.587678  -1.5780946  0.3      ]
 [ 8.668846   8.547335  -1.5750242  0.36     ]
 [ 8.708334   8.533     -1.5780946  0.35     ]
 [ 8.752992   8.49586   -1.5791179  0.37     ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]


In [31]:
xs = np.unique(voxel_coords[:,0])
ys = np.unique(voxel_coords[:,1])
zs = np.unique(voxel_coords[:,2])
print("voxel_coords unique:", len(xs), len(ys), len(zs))

print("gt_boxes:", gt_boxes[3])

voxel_coords unique: 1 187 224
gt_boxes: [25.187332  24.808327  -0.5687321  3.520522   1.6272181  1.4839411
  0.9085964  1.       ]
